# Basic plotting with plotly

In [1]:
from pathlib import Path

import arcadia_pycolor as apc
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from natsort import natsorted

from growth_curve_calculator import parse_spectramax_xml

### Sample absorption measurement heatmap

In [2]:
# Parse an XML file
example_data_directory = Path("../../growth_curve_calculator/tests/example_data")
xml_filepath = example_data_directory / "sample_endpoints_3.xml"
microplate_data_list = parse_spectramax_xml(xml_filepath)
plate_data = microplate_data_list[0]
# plate_data.measurements.head()

# Transform OD measurements into a format supported by seaborn heatmap
measurements = plate_data.measurements.pivot(
    index="well_row",
    columns="well_column",
    values="value",
)
measurements

well_column,1,2,3,4
well_row,,,,
A,0.112,0.445,0.122,0.112
B,0.107,0.641,0.157,0.167
C,0.092,0.343,0.138,0.118
D,0.123,0.440,0.153,0.117


In [3]:
# Create heatmap
fig = px.imshow(
    measurements,
    text_auto=True,
    aspect="auto",
    color_continuous_scale="RdBu_r",
)
fig.update_layout(
    width=500,
)
fig.show()

### Sample spectral scan

In [4]:
# Parse an XML file
xml_filepath = example_data_directory / "sample_spectrum_scans.xml"
microplate_data_list = parse_spectramax_xml(xml_filepath)
plate_data = microplate_data_list[9]
plate_data.measurements.head()

,well_row,well_column,well_id,value,excitation_nm,emission_nm
0,A,1,A01,4416022.0,350.0,570.0
1,A,2,A02,4134762.0,350.0,570.0
2,A,3,A03,4024690.0,350.0,570.0
3,A,4,A04,6265246.0,350.0,570.0
4,A,5,A05,4405988.0,350.0,570.0


In [5]:
# Sort out rows and columns
well_rows = natsorted(plate_data.measurements["well_row"].unique().tolist())  # type: ignore
well_cols = natsorted(plate_data.measurements["well_column"].unique().tolist())  # type: ignore
subplot_rows = {well_row: i + 1 for i, well_row in enumerate(well_rows)}
subplot_cols = {organism_id: i + 1 for i, organism_id in enumerate(well_cols)}

# Create figure
num_rows = len(subplot_rows)
num_cols = len(subplot_cols)
fig = make_subplots(
    rows=num_rows,
    cols=num_cols,
    shared_xaxes=True,
    shared_yaxes=True,
    row_titles=well_rows,
    column_titles=well_cols,
    x_title="Excitation wavelength (nm)",
)

for well_id, group in plate_data.measurements.groupby("well_id"):  # type: ignore
    trace = go.Scatter(
        x=group["excitation_nm"],
        y=group["value"],
        marker={"color": "blue"},
        mode="lines",
        showlegend=False,
    )

    well_row = well_id[0]
    well_col = int(well_id[1:])
    fig.add_trace(
        trace,
        row=subplot_rows[well_row],
        col=subplot_cols[well_col],
    )
fig.update_layout(
    width=900,
    height=600,
    template="plotly_white",
)
fig.show()